# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
# importing needed libraries for this lab
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
# with reset_index()
purchase_per_customer = data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index()
purchase_per_customer.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [5]:
# with as_index argument in .groupby method
purchase_per_customer2 = data.groupby(['CustomerID','ProductName'],as_index=False)['Quantity'].sum()
purchase_per_customer2.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [6]:
# with fillna(0) method --> 0 in float format
# with Total (All) column (margins=True)
product_by_customer_with_total = purchase_per_customer.pivot_table(index='ProductName',columns="CustomerID",
                                     values="Quantity",aggfunc=np.sum, margins=True).fillna(0)
display(product_by_customer_with_total.head())

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,All
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1918
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2228
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,1880
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0,1738
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2035


In [7]:
# with fill_value=0 argument of pd.pivot_table method --> 0 in int format
# without Total (All) column (margins=True)
product_by_customer = data.pivot_table(index='ProductName',columns="CustomerID",
                                     values="Quantity",aggfunc=np.sum,fill_value=0)
product_by_customer.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [8]:
transposed = product_by_customer.T
transposed.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
200,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
264,0,0,0,0,0,1,1,0,0,0,...,0,0,0,1,0,0,0,0,0,0
356,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
412,0,0,0,0,1,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [9]:
transposed_with_total = product_by_customer_with_total.T
transposed_with_total.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow,All
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,63.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,77.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,65.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,67.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.0


In [10]:
squareform_distance_matrix = squareform(pdist(transposed, 'euclidean'))
squareform_distance_matrix

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [11]:
squareform_distance_matrix_with_total = squareform(pdist(transposed_with_total, 'euclidean'))
squareform_distance_matrix_with_total

array([[0.00000000e+00, 1.83847763e+01, 1.06770783e+01, ...,
        2.00132206e+03, 1.77691868e+03, 8.86422779e+05],
       [1.83847763e+01, 0.00000000e+00, 1.67928556e+01, ...,
        1.98742748e+03, 1.76302808e+03, 8.86408764e+05],
       [1.06770783e+01, 1.67928556e+01, 0.00000000e+00, ...,
        1.99924886e+03, 1.77493549e+03, 8.86420779e+05],
       ...,
       [2.00132206e+03, 1.98742748e+03, 1.99924886e+03, ...,
        0.00000000e+00, 3.77491722e+02, 8.84433559e+05],
       [1.77691868e+03, 1.76302808e+03, 1.77493549e+03, ...,
        3.77491722e+02, 0.00000000e+00, 8.84658836e+05],
       [8.86422779e+05, 8.86408764e+05, 8.86420779e+05, ...,
        8.84433559e+05, 8.84658836e+05, 0.00000000e+00]])

In [12]:
distances_customers_with_total = pd.DataFrame(1/(1 +squareform_distance_matrix_with_total),
            index=product_by_customer_with_total.columns, columns=product_by_customer_with_total.columns)
distances_customers_with_total.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,All
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.051587,0.085638,0.077421,0.061859,0.080070,0.050740,0.081800,0.078977,0.075956,...,0.000634,0.000587,0.000726,0.000570,0.000605,0.000588,0.000587,0.000499,0.000562,0.000001
200,0.051587,1.000000,0.056202,0.059936,0.070468,0.057168,0.075956,0.047506,0.058185,0.060167,...,0.000640,0.000592,0.000733,0.000575,0.000610,0.000592,0.000592,0.000503,0.000567,0.000001
264,0.085638,0.056202,1.000000,0.078977,0.067640,0.080070,0.055647,0.076435,0.081800,0.077421,...,0.000635,0.000588,0.000727,0.000571,0.000606,0.000588,0.000588,0.000500,0.000563,0.000001
356,0.077421,0.059936,0.078977,1.000000,0.069010,0.077929,0.059261,0.073265,0.076923,0.075956,...,0.000636,0.000589,0.000728,0.000572,0.000607,0.000589,0.000588,0.000500,0.000564,0.000001
412,0.061859,0.070468,0.067640,0.069010,1.000000,0.068314,0.071626,0.057568,0.067975,0.070468,...,0.000638,0.000591,0.000731,0.000574,0.000609,0.000591,0.000591,0.000502,0.000566,0.000001


In [13]:
distances_customers = pd.DataFrame(1/(1 +squareform_distance_matrix),
            index=product_by_customer.columns, columns=product_by_customer.columns)
distances_customers.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [14]:
# checking we get the same results in both ways --> The figures are different, but they respect the same order
print(distances_customers_with_total[33].head(6))
print(distances_customers[33].head(6))

CustomerID
33     1.000000
200    0.051587
264    0.085638
356    0.077421
412    0.061859
464    0.080070
Name: 33, dtype: float64
CustomerID
33     1.000000
200    0.077421
264    0.087047
356    0.081800
412    0.080634
464    0.082709
Name: 33, dtype: float64


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [15]:
# Top 5 most similar customers for client ID 33 
similarities_client33 = distances_customers[33].sort_values(ascending=False)[1:6]
similarities_client33

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

In [16]:
# Top 5 most similar customers for client ID 200
similarities_client200 = distances_customers[200].sort_values(ascending=False)[1:]
similarities_client200_tab = pd.DataFrame(similarities_client200.reset_index().head())
similarities_client200_tab

,CustomerID,200
0,3317,0.084959
1,1072,0.083972
2,3535,0.082403
3,1920,0.082100
4,3909,0.080920


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [17]:
records_similarities_client33 = purchase_per_customer[purchase_per_customer["CustomerID"].isin(similarities_client33.index)]
records_similarities_client33.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


In [18]:
similarities_client200_tab.CustomerID

0    3317
1    1072
2    3535
3    1920
4    3909
Name: CustomerID, dtype: int64

In [19]:
records_similarities_client200 = purchase_per_customer[purchase_per_customer["CustomerID"].isin(similarities_client200_tab.CustomerID)]
records_similarities_client200.head()

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [20]:
products_similarities_client200 = records_similarities_client200.groupby(['ProductName'],
                        as_index=False)['Quantity'].sum().sort_values(by='Quantity', ascending=False)
# in order not to bore neither bother our TAs, let's only show the top similar/recommended products (>2 similarities)
products_similarities_client200[products_similarities_client200.Quantity > 2 ]

,ProductName,Quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3
88,Lamb - Ground,3
185,Wanton Wrap,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [21]:
# first we merge ranked products data frame with the customer product matrix on the ProductName field
# we need only the chosen client ID in the matrix in order to filter then
recommended_products_client200 = pd.merge(products_similarities_client200,product_by_customer[200],
                            left_on='ProductName',right_on='ProductName',how='left')
recommended_products_client200.head()

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,4,0
1,Muffin - Carrot Individual Wrap,3,0
2,Bay Leaf,3,0
3,Pork - Kidney,3,0
4,"Pepper - Black, Whole",3,1


In [22]:
# then we filter only by products not yet purchased by chosen customer (client 200)
recommended_new_products_client200 = recommended_products_client200[recommended_products_client200[200] == 0]
recommended_new_products_client200.head()

,ProductName,Quantity,200
0,Soup - Campbells Bean Medley,4,0
1,Muffin - Carrot Individual Wrap,3,0
2,Bay Leaf,3,0
3,Pork - Kidney,3,0
6,Wanton Wrap,3,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

### I think we should add these previous steps in order to automate all the process from raw data to neeeded outputs:

In [23]:
# step 0 --> loading CSV and get the raw dataframe
def csv_to_df(file_path):
    return pd.DataFrame(pd.read_csv(file_path))
file_path = '../data/customer_product_sales.csv'
data = csv_to_df(file_path)
data.head() # to check the function

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [24]:
# step 1 --> convert raw dataframe into a purchase_per_customer dataframe
def purchase_per_customer(data):
    return pd.DataFrame(data.groupby(['CustomerID','ProductName'])['Quantity'].sum().reset_index())
purchase_per_customer = purchase_per_customer(data)
purchase_per_customer.head() # to check the function

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


In [25]:
# step 2 --> convert raw dataframe into a products_by_customers matrix (without total)
def products_by_customers(data):
    return data.pivot_table(index='ProductName',columns="CustomerID",
                           values="Quantity",aggfunc=np.sum,fill_value=0)
products_by_customers = products_by_customers(data)
products_by_customers.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


In [26]:
# step 3 --> Create a customer similarity matrix with squareform and pdist.
# here we use Euclidean distance, but in case we want to change it in the future.
def distances_customers(products_by_customers,distance_type):
    return pd.DataFrame(1/(1 +squareform(pdist(products_by_customers.T, distance_type))),
            index=products_by_customers.columns, columns=products_by_customers.columns)
distance_type = 'euclidean'
distances_customers = distances_customers(products_by_customers,distance_type)
distances_customers.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


In [27]:
# step 4 --> Top N most similar customers for a specific client ID
# here Top 5, but in case we want to change N to a larger similar group of similar customers in the future
def topN_similar(clientID,n):
    return pd.DataFrame(distances_customers[clientID].sort_values(ascending=False)[1:n+1]).reset_index()
clientID = 200
n = 5
topN_similar(clientID,n) # to check the function

,CustomerID,200
0,3317,0.084959
1,1072,0.083972
2,3535,0.082403
3,1920,0.082100
4,3909,0.080920


In [28]:
# step 5 --> get the products purchased records from a specific customers list
# here Top 5 similars customers, but in case we need the function for another kind of condition in the future
def purchased_products(condition):
    return purchase_per_customer[purchase_per_customer['CustomerID'].isin(condition.CustomerID)]
condition = topN_similar(clientID,n)
purchased_products_df = purchased_products(condition)
purchased_products_df.head() # to check the function

,CustomerID,ProductName,Quantity
1082,1072,"Appetizer - Mini Egg Roll, Shrimp",1
1083,1072,Appetizer - Sausage Rolls,1
1084,1072,"Beans - Kidney, Canned",1
1085,1072,Beef - Montreal Smoked Brisket,1
1086,1072,Beef - Striploin Aa,1


In [29]:
# step 6 --> Most purchased products among a specific customers list
# here Top 5 similars customers, but in case we need the function for another kind of condition in the future
def most_purchased_products(purchased_products_df):
    return purchased_products_df.groupby(['ProductName'],
                        as_index=False)['Quantity'].sum().sort_values(by='Quantity', ascending=False)
most_purchased_products_df = most_purchased_products(purchased_products_df)
most_purchased_products_df.head() # to check the function

,ProductName,Quantity
159,Soup - Campbells Bean Medley,4
103,Muffin - Carrot Individual Wrap,3
8,Bay Leaf,3
136,Pork - Kidney,3
129,"Pepper - Black, Whole",3


In [30]:
# step 7 --> Recommend only new products for chosen clientID (not purchased yet) 
# and recommend top N most purchased products among must purchased products from a customer list (set by a condition)
# here Top 5 products, but in case we want to change x to a larger set of products in the future
def recommend_x_newproducts(clientID,x,condition):
    recommended_products = pd.merge(pd.DataFrame(most_purchased_products(purchased_products(condition))),
                                    pd.DataFrame(products_by_customers[clientID]), 
                                    left_on='ProductName',right_on='ProductName',how='left')
    return list(recommended_products[recommended_products[clientID] == 0][:x]['ProductName'])
x=5
recommend_x_newproducts = recommend_x_newproducts(clientID,x,condition)
recommend_x_newproducts

['Soup - Campbells Bean Medley',
 'Muffin - Carrot Individual Wrap',
 'Bay Leaf',
 'Pork - Kidney',
 'Wanton Wrap']

### we want to automate the process and be able to update the dict along the time so our recommendations can evolve

In [31]:
# creating a list of unique CustomerIDs to iterate over
def unique_customerIDs_list(data): 
    return data['CustomerID'].unique().tolist()
clients_list = unique_customerIDs_list(data)
len(clients_list) # checking 

1000

In [32]:
# creating a function that will generate an updated recommendations dict
def recommendations_xnew_topNsimilar(clients_list,x,n):
    # creating empty dictionary that will hold the recommendations for each customer
    recommendations_dict = {}
    # Iterating over customer list performing from step 4 to step 7 for each & appending results to the dict
    for c in clients_list:
        cond = topN_similar(c,n)
        recommended_products = pd.merge(pd.DataFrame(most_purchased_products(purchased_products(cond))),
                                    pd.DataFrame(products_by_customers[clientID]), 
                                    left_on='ProductName',right_on='ProductName',how='left')
        recommendations_dict[c] = list(recommended_products[recommended_products[clientID] == 0][:x]['ProductName'])
    return recommendations_dict
x, n = 5, 5
recommendations_dict = recommendations_xnew_topNsimilar(clients_list,x,n)
print('recos for client 33:',recommendations_dict[33]) # to check
print('recos for client 1920:',recommendations_dict[1920]) # to check
print('recos for client 3909:',recommendations_dict[3909]) # to check

recos for client 33: ['Butter - Unsalted', 'Wine - Ej Gallo Sierra Valley', 'Towels - Paper / Kraft', 'Soup - Campbells Bean Medley', 'Wine - Blue Nun Qualitatswein']
recos for client 1920: ['Wine - Blue Nun Qualitatswein', 'Sardines', 'Water, Tap', 'Beef Wellington', 'Wine - Chardonnay South']
recos for client 3909: ['Soupfoamcont12oz 112con', 'Fondant - Icing', 'Wine - Magnotta, Merlot Sr Vqa', 'Soup - Campbells Bean Medley', 'Cod - Black Whole Fillet']


### Main - from raw data to recommendations outputs

In [33]:
# file_path = '../data/customer_product_sales.csv'
# data = csv_to_df(file_path)
# clients_list = unique_customerIDs_list(data)
# x, n = 5, 5
# recommendations_dict = recommendations_xnew_topNsimilar(clients_list,x,n)

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [34]:
# let's build a function to automate all the process:
def df_top5recom_top5similar(recommendations_dict):
    df_recom = pd.DataFrame.from_dict(recommendations_dict).T.reset_index()
    df_recom.columns= ['ClientID','Reco1','Reco2','Reco3','Reco4','Reco5']
    return df_recom
df_top5recom_top5similar = df_top5recom_top5similar(recommendations_dict)
df_top5recom_top5similar.head()

,ClientID,Reco1,Reco2,Reco3,Reco4,Reco5
0,61288,Beef - Rib Eye Aaa,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,"Lamb - Pieces, Diced"
1,77352,Wine - Valpolicella Masi,Fuji Apples,Ocean Spray - Ruby Red,Tuna - Salad Premix,Chicken - Soup Base
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,"Cheese - Brie,danish",Cookie - Dough Variety,Tuna - Salad Premix
3,23548,Tea - English Breakfast,Squid - Tubes / Tenticles 10/20,Spinach - Baby,Jolt Cola - Electric Blue,Sprouts - Baby Pea Tendrils
4,78981,Isomalt,Vanilla Beans,Lettuce - Frisee,Grouper - Fresh,Corn Meal


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [39]:
def recommendations_xnew_topNsimilar_distance_type(clients_list,x,n,distance_type):
    # creating empty dictionary that will hold the recommendations for each customer
    recommendations_dict = {}
    # Iterating over customer list performing from step 4 to step 7 for each & appending results to the dict
    for c in clients_list:
        distances_customers = distances_customers(products_by_customers,distance_type)
        cond = topN_similar(c,n)
        recommended_products = pd.merge(pd.DataFrame(most_purchased_products(purchased_products(cond))),
                                    pd.DataFrame(products_by_customers[clientID]), 
                                    left_on='ProductName',right_on='ProductName',how='left')
        recommendations_dict[c] = list(recommended_products[recommended_products[clientID] == 0][:x]['ProductName'])
    return recommendations_dict

recommendations_dict_cityblock = recommendations_xnew_topNsimilar_distance_type(clients_list,5,5,'cityblock')
df_top5recom_top5similar_cityblock = df_top5recom_top5similar(recommendations_dict_cityblock)
df_top5recom_top5similar_cityblock.head()

UnboundLocalError: local variable 'distances_customers' referenced before assignment

In [ ]:
# file_path = '../data/customer_product_sales.csv'
# data = csv_to_df(file_path)
# # purchase_per_customer = purchase_per_customer(data)
# # products_by_customers = products_by_customers(data)
# distance_type = 'euclidean'
# distances_customers = distances_customers(products_by_customers,distance_type)
# clientID = 200
# n = 5
# condition = topN_similar(clientID,n)
# purchased_products_df = purchased_products(condition)
# most_purchased_products_df = most_purchased_products(purchased_products_df)
# x=5
# def recommendations_xnew_topNsimilar(clients_list,x,n):
#     # creating empty dictionary that will hold the recommendations for each customer
#     recommendations_dict = {}
#     # Iterating over customer list performing from step 4 to step 7 for each & appending results to the dict
#     for c in clients_list:
#         cond = topN_similar(c,n)
#         recommended_products = pd.merge(pd.DataFrame(most_purchased_products(purchased_products(cond))),
#                                     pd.DataFrame(products_by_customers[clientID]), 
#                                     left_on='ProductName',right_on='ProductName',how='left')
#         recommendations_dict[c] = list(recommended_products[recommended_products[clientID] == 0][:x]['ProductName'])
#     return recommendations_dict
# recommend_x_newproducts = recommend_x_newproducts(clientID,x,condition)